# 📊 IORAG Experiment Notebook

> **Data Management Guide**: 本notebook使用标准化的路径配置。请阅读 [README_DATA.md](../../README_DATA.md) 了解数据管理最佳实践。

## 📁 目录结构
```
DarshanRAG/
├── data/                    # 数据文件（不提交git）
│   ├── raw/                 # 原始日志
│   ├── parsed/              # 解析后的日志
│   ├── archives/            # 归档文件
│   └── examples/            # 示例文件
├── knowledge_graphs/        # KG文件（不提交git）
├── experiments/
│   ├── notebooks/           # 📍 当前位置
│   ├── scripts/             # 实验脚本
│   ├── results/             # 结果输出（不提交git）
│   └── storage/             # RAG存储（不提交git）
```

## ⚠️ 重要提示
1. **不要硬编码路径** - 使用 `config_paths` 模块
2. **API Key** - 使用环境变量，不要硬编码在代码中
3. **大文件** - 所有大文件自动被git忽略，不会提交

## 📝 Git提交时推荐的完整流程
```
# Step 1: 检查状态
git status

# Step 2: 添加所有更改（.gitignore会自动过滤大文件）
git add -A

# Step 3: 检查暂存区（确保没有大文件）
git status
git diff --cached --stat

# Step 4: 如果一切正常，提交
git commit -m "Finish KG building"

# Step 5: 推送
git push origin main
```

## 🚀 Setup

In [ ]:
# ============================================================
# API Key 配置 - 使用环境变量（推荐）
# ============================================================
import os

# ⚠️ 不要硬编码API Key！使用环境变量
# 方式1: 从环境变量读取（推荐）
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY 环境变量未设置")
    print("请在终端中运行: export OPENAI_API_KEY='your-key-here'")
    # 或者临时设置（仅用于测试，不要提交到git）
    # os.environ["OPENAI_API_KEY"] = "your-key-here"
else:
    print("✅ OPENAI_API_KEY 已加载")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

✅ OPENAI_API_KEY 已加载


In [13]:
# ============================================================
# 路径配置 - 使用 config_paths 模块
# ============================================================
import sys
from pathlib import Path

# 添加experiments目录到Python路径
project_root = Path('/users/Minqiu/DarshanRAG')
sys.path.insert(0, str(project_root / 'experiments'))

from config_paths import PATHS, get_parsed_log_path, get_kg_path, get_storage_dir

# 配置实验数据路径
EXPERIMENT_DATE = '2025-1-1'
PARSED_LOGS = get_parsed_log_path(EXPERIMENT_DATE)
KG_FILE = get_kg_path('kg', EXPERIMENT_DATE)
STORAGE_DIR = get_storage_dir(f'lightrag_{EXPERIMENT_DATE.replace("-", "_")}')

# 确保结果输出目录存在
RESULTS_DIR = PATHS['results'] / f'iorag_{EXPERIMENT_DATE}'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 路径配置完成:")
print(f"  📂 Parsed logs: {PARSED_LOGS}")
print(f"  🕸️  KG file: {KG_FILE}")
print(f"  💾 Storage: {STORAGE_DIR}")
print(f"  📊 Results: {RESULTS_DIR}")

✅ 路径配置完成:
  📂 Parsed logs: /users/Minqiu/DarshanRAG/data/parsed/parsed-logs-2025-1-1
  🕸️  KG file: /users/Minqiu/DarshanRAG/knowledge_graphs/kg_2025-1-1.json
  💾 Storage: /users/Minqiu/DarshanRAG/experiments/storage/lightrag_2025_1_1
  📊 Results: /users/Minqiu/DarshanRAG/experiments/results/iorag_2025-1-1


# **Parse log**

PS. 要么用agent重新编译并支持 bzip2 支持3.5.0；要么装3.4.1

In [7]:
!chmod +x /users/Minqiu/DarshanRAG/experiments/scripts/unpack-darshan-logs.sh
!/users/Minqiu/DarshanRAG/experiments/scripts/unpack-darshan-logs.sh /users/Minqiu/polaris-darshan-logs-25-1

Extracting: /users/Minqiu/polaris-darshan-logs-25-1/8/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/21/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/24/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/2/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/10/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/11/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/26/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/15/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/20/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/30/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/19/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/1/logs.tar.gz
Done: /users/Minqiu/polaris-darshan-logs-25-1/8/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/13/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/27/logs.tar.gz
Extra

In [2]:
# 假设 parent_dir 是你想要遍历的父文件夹路径
parent_dir = '/users/Minqiu/polaris-darshan-logs-25-1'  # 修改为你的实际父文件夹

import os
import subprocess
from pathlib import Path

parsed_root = Path('~/parsed-logs-25-1').expanduser()
parsed_root.mkdir(parents=True, exist_ok=True)

darshan_logs = []
for root, dirs, files in os.walk(parent_dir):
    for file in files:
        if file.endswith('.darshan') or '.darshan.' in file:
            fullpath = os.path.join(root, file)
            # 生成相对于parent_dir的路径
            rel = os.path.relpath(fullpath, parent_dir)
            output_dir = parsed_root / Path(rel).parent
            output_dir.mkdir(parents=True, exist_ok=True)
            out_txt = output_dir / (Path(file).stem + '.txt')
            darshan_logs.append((fullpath, out_txt))

from tqdm import tqdm

# 统计成功和失败的数量
success_count = 0
fail_count = 0

for in_log, out_txt in tqdm(darshan_logs, desc="Parsing darshan logs"):
    # 使用 subprocess 运行命令并检查返回码
    result = subprocess.run(
        ['darshan-parser', in_log],
        capture_output=True,
        text=True
    )
    
    # 只在命令成功时写入文件
    if result.returncode == 0:
        with open(out_txt, 'w') as f:
            f.write(result.stdout)
        success_count += 1
    else:
        # 命令失败，不创建空文件
        fail_count += 1
        # 可选：记录失败的日志
        # print(f"Failed to parse {in_log}: {result.stderr.strip()}")

print(f"\n✅ 解析完成: 成功 {success_count} 个, 失败 {fail_count} 个")

Parsing darshan logs: 100%|██████████████████████████████████████████████████████| 24908/24908 [06:21<00:00, 65.32it/s]


✅ 解析完成: 成功 24392 个, 失败 516 个


可以加上失败原因

# **Process log (creating original_name_signals_v2.2.txt)**

In [ ]:
# 1. 处理单个文件
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/process_darshan_signals_v2.4.py \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt \
    -o /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Processing: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt
  -> Output: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Processing complete!


In [ ]:
# 2. 处理文件夹
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/process_darshan_signals_v2.4.py \
    /path/to/logs/ \
    -o ./my_signals/

# **Building KG (creating original_name_kg_v1.txt)**

In [ ]:
!python /users/Minqiu/DarshanRAG/experiments/darshan_kg_builder_v2.1.py \
    -i /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt \
    -o /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.1.json

Parsing single file: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Building Knowledge Graph...

Knowledge Graph Statistics:
Total entities: 68
Total relationships: 99

Entity breakdown:
  APPLICATION: 1
  FILE: 22
  FILESYSTEM: 1
  JOB: 1
  MODULE: 3
  RECORD: 40

Output saved to: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v2.1.json


# **Extracting Summary and updating KG**

In [19]:
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/generate_descriptions_v3.py \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.1.json \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v3.json


Reading /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.1.json...
Processing 68 entities and 99 relationships...
Writing to /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v3.json...
Done!

使用统计报告

【1】实体模板中永远没有匹配到的属性（总是 N/A）:
----------------------------------------------------------------------

  APPLICATION:
    - nprocs (缺失1次)

  FILE:
    - rank (缺失22次)
    - is_shared (缺失22次)

  JOB:
    - nnodes (缺失1次)
    - exe (缺失1次)

  RECORD:
    - io_start_ts (缺失40次)
    - seq_write_ratio (缺失40次)
    - consec_write_ratio (缺失40次)

【2】关系模板中永远没有匹配到的属性（总是 N/A）:
----------------------------------------------------------------------

  MODULE→RECORD:
    - operation_types (缺失40次)


【3】实体 JSON 中永远没有用到的属性:
----------------------------------------------------------------------

  FILE (1个未使用属性):
    - file_path_norm

  JOB (6个未使用属性):
    - job_io_summary
    - mount_table
    - total_bytes_read
    - total_bytes_written
    - total_reads
    - total_writes

  MODU

# **Embedding KG**

# **Visualize IORAG**

*3. 浏览器打开 http://localhost:9621*

*就可以用Web UI查询和可视化图谱了*

# **Query Testing**